# Coursework Assignment: KNN on Wine DataSet

```
University of London
BSc in Computer Science
CM3015, Machine Learning and Neural Networks
Hudson Leonardo MENDES
hlm12@student.london.ac.uk
```


# 1. Exploratory Data Analysis


In [ ]:
%matplotlib inline
# import modules
from sklearn import datasets import numpy as np
import matplotlib.pyplot as plt import pandas as pd
# load data
wine=datasets.load_wine() #print(wine.DESCR)
# this dataset has 13 features, we will only choose a subset of these
df_wine = pd.DataFrame(wine.data, columns = wine.feature_names )
selected_features = ['alcohol','flavanoids','color_intensity','ash']
# extract the data as numpy arrays of features, X, and target, y
X = df_wine[selected_features].values
y = wine.target

## 1.1. Visualising the Data


In [ ]:
def plot_wine_dataset_scatter(df):
    pass

In [ ]:
plot_wine_dataset_scatter(df=X)

## 1.2. Exploratory Data Analysis under Noise


In [ ]:
r = 42
np.random.seed(r)
XN = X+np.random.normal(0,0.6,X.shape)

In [ ]:
plot_wine_dataset_scatter(df=XN)

## 1.3. (Q1) Exploratory Data Analysis

**Question:** If you were to build a classifier using only two of the available features, which ones would you choose any why?


**Answer:** `<answer>`


## 1.4. (Q2) Data with Noise

**Question:** What do you observe by plotting the data without noise compared to plotting data with added Gaussian noise?


**Answer:** `<answer>`


# 2. Classifier Implementation


In [ ]:
import numpy as np
from typing import List, Any

class KnnIndex:
    documents: List[Any]
    vectors: np.array

    def __init__(self, documents, targets):
        self.documents = documents
        self.targets = targets
        self.vectors = None #TODO: implement

In [ ]:
def knn_classification(X_train, y_train, X_test, options):
    """
    Performs a k-NN classification given a set of data.

    :param X: documents representing the training data
    :param y: training target classes/labels
    :param X_: testing documents
    :param y_: predicted outputs
    """
    index = KnnIndex(documents=X_train, targets=y_train)
    return index.search(X_test)

In [ ]:
train_proportion = 0.8
train_size = int(X.shape[0] * train_proportion)
X1, X2 = X[:train_size], X[train_size:]
y1, y2 = y[:train_size], y[train_size:]
(X1.shape, X2.shape, y1.shape, y2.shape)

In [ ]:
knn_classification(
    X_train=X1,
    y_train=y1,
    X_test=X2, options={})

# 3. Classifier Evaluation


In [ ]:
def plot_metrics(X, y, X_, y_, options):
    pass

In [ ]:
plot_metrics(X)

## 3.1. Nested Cross-Validation


In [ ]:
import numpy as np
from typing import List

def nested_cross_validation(
        X_full: np.array,
        y_full: np.array,
        n_folds: int,
        distances: List[str],
        random_state: int,
    ):
    pass

In [ ]:
accuracies_fold_X = nested_cross_validation(
    X_full=X,
    y_full=y,
    n_knn=list(range(1, 11)),
    n_folds=5,
    distances=["euclidean", "manhattan", "cosine"],
    random_state=42
)

In [ ]:
accuracies_fold_XN = nested_cross_validation(
    X_full=XN,
    y_full=y,
    n_knn=list(range(1, 11)),
    n_folds=5,
    distances=["euclidean", "manhattan", "cosine"],
    random_state=42
)

# 5. Summary of Results


## 5.1. Consolidated Results


**Results on Non-Noisy Data**

`<table>`

**Results on Noisy Data**

`<table>`

# 6. More questions


## 6.1. [Q3] Influence of Noise

**Question:** (a) Do the best parameters change when noise is added to the data? (b) Can you say that one parameter choice is better regardless of the data used?

**Answer:** `<answer>`

## 6.2. [Q4] Tie Break

**Question:** Assume that you have selected the number of neighbours to be an even number (e.g.: 2). For one of the neighbours, the suggested class is 1, and for the other neighbours, the suggested class is 2. How would you  break the tie?

**Answer:** `<answer>`

## 6.3. [Q5] Beyond  Wine

**Question:** If you were to run your k-NN algorithm on a new dataset (e.g.: the breast cancer dataset, or iris), what would you need to take in consideration? Outline any changes that might be needed to your code.

**Answer:** `<answer>`